In [9]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.auto import tqdm

In [3]:
df_subm = pd.read_csv('./sample_submission.csv')

In [6]:
# 예측 대상 파일 목록 생성
list_c = glob('./test/C/*.csv')
list_d = glob('./test/D/*.csv')

In [ ]:
# 조건 파라미터 설정
nr = 3 # 이동 평균 간격

nl11 = 3 # 시점 조건1의 세부 조건1의 지속 시간
nl12 = 5 # 시점 조건1의 세부 조건2의 지속 시간
nl2  = 2 # 시점 조건2의 지속 시간
nl3  = 6 # 시점 조건3의 지속 시간

th11 = 0.975 # 시점 조건1의 세부 조건1의 임계값
th12 = 0.975 # 시점 조건1의 세부 조건2의 임계값
th2  = 0.95  # 시점 조건2의 임계값
th3  = 1.0   # 시점 조건3의 임계값

nlp  = 60    # 지점 조건의 과거 이동 평균 계산 일수
thp  = 0.975  # 지점 조건의 임계값

sus_c = [] 
sus_d = [] 

In [7]:
# 조건 파라미터 설정
win = 3 # 이동 평균 간격

m_1 = 3 # 시점 조건1의 세부 조건1의 지속 시간
m_2 = 5 # 시점 조건1의 세부 조건2의 지속 시간
m_3  = 2 # 시점 조건2의 지속 시간
m_4  = 6 # 시점 조건3의 지속 시간

th_1 = 0.975 # 시점 조건1의 세부 조건1의 임계값
th_2 = 0.975 # 시점 조건1의 세부 조건2의 임계값
th_3  = 0.95  # 시점 조건2의 임계값
th_4  = 1.0   # 시점 조건3의 임계값

m_5  = 60    # 지점 조건의 과거 이동 평균 계산 일수
th_5  = 0.975  # 지점 조건의 임계값

sus_c = [] 
sus_d = [] 

In [10]:
# Network C
for i in tqdm(range(len(list_c))):
    df_c_ = pd.read_csv(list_c[i]).iloc[-100:]

    # 파생 변수 생성
    df_c_['_Qin']   = df_c_['Q1'].copy()
    df_c_['_Qout']  = df_c_.filter(regex='Q[2-8]+$').sum(axis=1)
    df_c_['_Qratio'] = df_c_['_Qout']/df_c_['_Qin']
    df_c_['_Qratio2'] = df_c_['_Qout'].rolling(win).mean() / df_c_['_Qin'].rolling(win).mean()

    # T+1 시점의 이상 여부 예측
    cond_c1 = (df_c_['_Qratio2'].iloc[-m_1:] < th_1).all()
    cond_c2 = (df_c_['_Qratio' ].iloc[-m_2:] < th_2).all()
    cond_c3  = (df_c_['_Qratio' ].iloc[-m_3: ] < th_3 ).all()
    cond_c4  =  df_c_['_Qratio' ].iloc[-m_4: ].max() < th_4
    if ((cond_c1 and cond_c2) or cond_c3) and cond_c4:
        sus_c.append(list_c[i])

        # 이상 지점 탐지를 위한 압력계별 변화율 계산 및 예측
        c_c1_sub = (df_c_['_Qratio2'] < th_1).rolling(m_1).sum() == m_1
        c_c2_sub = (df_c_['_Qratio']  < th_2).rolling(m_2).sum() == m_2
        c_c3_sub  = (df_c_['_Qratio']  < th_3 ).rolling(m_3 ).sum() == m_3

        # 해당 데이터 내 이상 지점 시작 시점 확인 및 부분 압력계 데이터 생성
        idx_start = df_c_[(c_c1_sub & c_c2_sub) | c_c3_sub].index.min()
        df_c_P = df_c_.filter(regex='P[0-9]+$')

        # 직전 30분 대비 이상 시점 평균 압력 변화율 계산
        mean_p = df_c_P.loc[idx_start:].mean() / df_c_P.loc[(idx_start-m_5):(idx_start-1)].mean() 

        # 이상 지점 확인 및 예측값 생성
        mean_p_rel = mean_p / mean_p.mean()
        tf_p_rel   = mean_p_rel < th_5

        if ~tf_p_rel.any():
            tf_p_rel = (mean_p_rel == mean_p_rel.min())

        val_pred = '[' + (tf_p_rel + 0).astype('string').str.cat(sep=', ') + ']'

        # 예측값 입력
        name_file = list_c[i][9:20]
        df_subm.loc[df_subm['ID'] == name_file, 'flag_list'] = val_pred
        print(name_file, val_pred)

print(f'Network C: Total {len(sus_c)} anomaly points')

  0%|          | 0/2920 [00:00<?, ?it/s]

TEST_C_0023 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_0038 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_0082 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_0165 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_0179 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_0264 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_0509 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_0724 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_0774 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_0917 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_0934 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_1238 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_1356 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_1520 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_1542 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_1582 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_1601 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_1636 [0, 0, 0, 0, 1, 0, 0, 0]
TEST_C_1654 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_1683 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_1763 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_1853 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_1889 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_1910 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_2152 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_2191 [0, 0, 0, 0, 0, 0, 1, 0]
TEST_C_2279 [0, 0, 0, 0, 0, 0, 1, 0]
T

In [ ]:
# Network D
for i in tqdm(range(len(list_d))):
    df_d_ = pd.read_csv(list_d[i]).iloc[-100:]

    # 파생 변수 생성
    df_d_['_Qin']   = df_d_['Q1'].copy()
    df_d_['_Qout']  = df_d_.filter(regex='Q[2-5]+$').sum(axis=1)
    df_d_['_Qratio'] = df_d_['_Qout']/df_d_['_Qin']
    df_d_['_Qratio2'] = df_d_['_Qout'].rolling(win).mean() / df_d_['_Qin'].rolling(win).mean()

    # T+1 시점의 이상 여부 예측
    cond_d1 = (df_d_['_Qratio2'].iloc[-m_1:] < th_1).all()
    cond_d2 = (df_d_['_Qratio' ].iloc[-m_2:] < th_2).all()
    cond_d3  = (df_d_['_Qratio' ].iloc[-m_3: ] < th_3 ).all()
    cond_d4  =  df_d_['_Qratio' ].iloc[-m_4: ].max() < th_4
    if ((cond_d1 and cond_d2) or cond_d3) and cond_d4:
        sus_d.append(list_d[i])

        # 이상 지점 탐지를 위한 압력계별 변화율 계산 및 예측
        c_d1_sub = (df_d_['_Qratio2'] < th_1).rolling(m_1).sum() == m_1
        c_d2_sub = (df_d_['_Qratio']  < th_2).rolling(m_2).sum() == m_2
        c_d3_sub  = (df_d_['_Qratio']  < th_3 ).rolling(m_3 ).sum() == m_3

        # 해당 데이터 내 이상 지점 시작 시점 확인 및 부분 압력계 데이터 생성
        idx_start = df_d_[(c_d1_sub & c_d2_sub) | c_d3_sub].index.min()
        df_d_P = df_d_.filter(regex='P[0-9]+$')

        # 직전 30분 대비 이상 시점 평균 압력 변화율 계산
        mean_p = df_d_P.loc[idx_start:].mean() / df_d_P.loc[(idx_start-m_5):(idx_start-1)].mean() 

        # 이상 지점 확인 및 예측값 생성
        mean_p_rel = mean_p / mean_p.mean()
        tf_p_rel   = mean_p_rel < th_5

        if ~tf_p_rel.any():
            tf_p_rel = (mean_p_rel == mean_p_rel.min())

        val_pred = '[' + (tf_p_rel + 0).astype('string').str.cat(sep=', ') + ']'

        # 예측값 입력
        name_file = list_d[i][9:20]
        df_subm.loc[df_subm['ID'] == name_file, 'flag_list'] = val_pred
        print(name_file, val_pred)

print(f'Network D: Total {len(sus_d)} anomaly points')


  0%|          | 0/2738 [00:00<?, ?it/s]

TEST_D_0024 [0, 0, 0, 0, 0, 1]
TEST_D_0033 [0, 0, 0, 0, 0, 1]
TEST_D_0060 [0, 0, 0, 0, 0, 1]
TEST_D_0082 [0, 0, 0, 0, 0, 1]
TEST_D_0092 [0, 0, 0, 0, 0, 1]
TEST_D_0107 [0, 0, 0, 0, 0, 1]
TEST_D_0210 [0, 0, 0, 0, 0, 1]
TEST_D_0216 [0, 0, 0, 0, 0, 1]
TEST_D_0229 [0, 0, 0, 0, 0, 1]
TEST_D_0468 [0, 0, 0, 0, 0, 1]
TEST_D_0760 [0, 0, 0, 0, 0, 1]
TEST_D_0783 [0, 0, 0, 0, 0, 1]
TEST_D_0870 [0, 0, 0, 0, 0, 1]
TEST_D_1016 [0, 0, 0, 0, 0, 1]
TEST_D_1029 [0, 0, 0, 0, 0, 1]
TEST_D_1091 [0, 0, 0, 0, 0, 1]
TEST_D_1119 [0, 0, 0, 0, 0, 1]
TEST_D_1192 [0, 0, 0, 0, 0, 1]
TEST_D_1198 [0, 0, 0, 0, 0, 1]


In [ ]:
df_subm['flag_list'].value_counts()

In [ ]:
df_subm.to_csv('./AIM_test.csv', index=False)